In [1]:
import time
from collections import Counter
from PIL import Image, ImageDraw
import os
import pyautogui
import arcade

In [2]:
l = 20
h = 14

base_x = 235
base_y = 5

step_x = 130
step_y = 65

box_size = 30

dict_swap_dofus_to_pos = {}
dict_swap_pos_to_dofus  = {}

In [3]:
for j in range(h):
    
    x_start = 13 - j
    
    for i in range(l):
        
        dict_swap_dofus_to_pos[(j, i, False)] = (x_start + i, i + j)
        dict_swap_dofus_to_pos[(j, i, True)] = (x_start + i, i + j + 1)
        
        dict_swap_pos_to_dofus[(x_start + i, i + j)] = (j, i, False)
        dict_swap_pos_to_dofus[(x_start + i, i + j + 1)] = (j, i, True)


In [4]:
print(dict_swap_dofus_to_pos)

print(dict_swap_dofus_to_pos[(0, 0, False)])
print(dict_swap_dofus_to_pos[(0, 0, True)])

print(dict_swap_dofus_to_pos[(1, 0, False)])
print(dict_swap_dofus_to_pos[(1, 0, True)])

print(dict_swap_dofus_to_pos[(1, 1, False)])
print(dict_swap_dofus_to_pos[(1, 1, True)])

{(0, 0, False): (13, 0), (0, 0, True): (13, 1), (0, 1, False): (14, 1), (0, 1, True): (14, 2), (0, 2, False): (15, 2), (0, 2, True): (15, 3), (0, 3, False): (16, 3), (0, 3, True): (16, 4), (0, 4, False): (17, 4), (0, 4, True): (17, 5), (0, 5, False): (18, 5), (0, 5, True): (18, 6), (0, 6, False): (19, 6), (0, 6, True): (19, 7), (0, 7, False): (20, 7), (0, 7, True): (20, 8), (0, 8, False): (21, 8), (0, 8, True): (21, 9), (0, 9, False): (22, 9), (0, 9, True): (22, 10), (0, 10, False): (23, 10), (0, 10, True): (23, 11), (0, 11, False): (24, 11), (0, 11, True): (24, 12), (0, 12, False): (25, 12), (0, 12, True): (25, 13), (0, 13, False): (26, 13), (0, 13, True): (26, 14), (0, 14, False): (27, 14), (0, 14, True): (27, 15), (0, 15, False): (28, 15), (0, 15, True): (28, 16), (0, 16, False): (29, 16), (0, 16, True): (29, 17), (0, 17, False): (30, 17), (0, 17, True): (30, 18), (0, 18, False): (31, 18), (0, 18, True): (31, 19), (0, 19, False): (32, 19), (0, 19, True): (32, 20), (1, 0, False): (12

In [5]:
def crop_image(image, rect, path):
    region = image.crop(rect)
    region.save(path)

In [6]:
def full_crop_image(image_name):
    img = Image.open(f"data/fight/{image_name}.png")
    os.mkdir(f"data/cropped/{image_name}")
    
    draw = ImageDraw.Draw(img)
    
    for i in range(h):
        for j in range(l):
            x = base_x + step_x * i
            y = base_y + step_y * j
            
            crop_image(img, (x, y, x + box_size, y + box_size), f"data/cropped/{image_name}/{image_name}-{i}-{j}-false.png")
            draw.rectangle((x, y, x + box_size, y + box_size), outline="red", width=2)
            
            x += step_x // 2
            y += step_y // 2
            crop_image(img, (x, y, x + box_size, y + box_size), f"data/cropped/{image_name}/{image_name}-{i}-{j}-true.png")
            draw.rectangle((x, y, x + box_size, y + box_size), outline="red", width=2)

    
    img.save(f"data/cropped_visu/{image_name}.png")
    
    img.close()

In [7]:
#folder_dir = "data/fight/"
#for images in os.listdir(folder_dir):
#    if images.endswith(".png"):
#        full_crop_image(images.split('.')[0])

In [8]:
def get_colors(path=None, img=None):
    
    if img is None:
        img = Image.open(path)
        
    pixels = list(img.getdata())

    colors_counter = Counter(pixels)
    colors_counter = sorted(colors_counter.items(), key=lambda x: x[1], reverse=True)

    color1, _ = colors_counter[0]    
    if len(colors_counter) > 1:
        color2, _ = colors_counter[1]
    else :
        color2 = (0, 0, 0)
        
    return color1, color2

In [9]:
list_color = {}

for image in os.listdir("./data/cropped/139"):
    if image.endswith(".png"):
        color = get_colors("./data/cropped/139/" + image)
        if color[0] not in list_color.keys():
            list_color[color[0]] = image
            print(f"{image} -> {color}")


139-8-6-false.png -> ((142, 134, 94, 255), (88, 83, 58, 255))
139-6-2-false.png -> ((0, 0, 0, 255), (0, 0, 0))
139-12-11-false.png -> ((88, 83, 58, 255), (89, 84, 59, 255))
139-7-9-true.png -> ((150, 142, 103, 255), (88, 83, 58, 255))
139-5-11-true.png -> ((218, 57, 45, 255), (230, 127, 46, 255))
139-2-12-true.png -> ((46, 54, 61, 255), (77, 77, 93, 255))
139-8-9-false.png -> ((251, 241, 191, 255), (46, 54, 61, 255))
139-6-17-false.png -> ((196, 19, 0, 255), (101, 11, 1, 255))


In [27]:
EMPTY_CELL = 0
WALL = 1
USABLE_CELL = 2
PLAYER = 3
MONSTER = 4
INVO = 5
BOSS = 6
UNDEFINED = -1

list_cell_color = {
    (0, 0, 0, 255): EMPTY_CELL,
    (142, 134, 94, 255): USABLE_CELL,
    (150, 142, 103, 255): USABLE_CELL,
    (85, 121, 56, 255): USABLE_CELL,
    (90, 125, 62, 255): USABLE_CELL,
    (88, 83, 58, 255): WALL,
    (218, 57, 45, 255): INVO,
    (46, 54, 61, 255): MONSTER,
    (251, 241, 191, 255): BOSS,
    (196, 19, 0, 255): PLAYER,
}


In [28]:
def get_image_type(path=None, img=None):
    
    if img is None:
        most = get_colors(path=path)
    else:
        most = get_colors(img=img)
    
    type = UNDEFINED
    
    if most[0] in list_cell_color.keys():
        type = list_cell_color[most[0]]
    else:
        print(most[0])
        
    if type == USABLE_CELL and most[1] == (46, 54, 61):
        type = MONSTER
        
    return type

In [35]:
def get_screen_info():
    
    dict_case = {}
    
    screen = pyautogui.screenshot()
    
    for pos in dict_swap_pos_to_dofus.keys():
        
        dofus_pos = dict_swap_pos_to_dofus[pos]
        x = base_x + step_x * dofus_pos[0]
        y = base_y + step_y * dofus_pos[1]
        
        if dofus_pos[2]:
            x += step_x // 2
            y += step_y // 2
        
        cropped = screen.crop((x, y, x + box_size, y + box_size))

        dict_case[pos] = get_image_type(img=cropped)

    return dict_case

In [32]:
class DofusStatusWindow(arcade.Window):
    
    def __init__(self):
        super().__init__(2000, 1500, "DofusStatus")
        arcade.set_background_color(arcade.color.BLACK)
        self.status = {}
        
    def draw_box(self, x, y, color):
        arcade.draw_rectangle_filled(x, 1500 - y, box_size // 2, box_size // 2, color)
        
    def on_draw(self):
        arcade.start_render()
        for pos in self.status.keys():
            if self.status[pos] != 0:

                dofus_pos = dict_swap_pos_to_dofus[pos]

                x = base_x + step_x * dofus_pos[0]
                y = base_y + step_y * dofus_pos[1]
                
                if dofus_pos[2]:
                    x += step_x // 2
                    y += step_y // 2
                
                x = x // 2
                y = y // 2
                
                if self.status[pos] == 1:
                    self.draw_box(x, y, arcade.color.GRAY)
                elif self.status[pos] == 2:
                    self.draw_box(x, y, arcade.color.GREEN)
                elif self.status[pos] == 3:
                    self.draw_box(x, y, arcade.color.BLUE)
                elif self.status[pos] == 4:
                    self.draw_box(x, y, arcade.color.RED)
                elif self.status[pos] == 5:
                    self.draw_box(x, y, arcade.color.YELLOW)
                elif self.status[pos] == 6:
                    self.draw_box(x, y, arcade.color.ORANGE)
                else:
                    self.draw_box(x, y, arcade.color.AQUA)

    def on_update(self, delta_time):
        time.sleep(1)
        self.status = get_screen_info()

window = DofusStatusWindow()
arcade.run()

(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(34, 34, 34, 255)
(34, 34, 34, 255)
(19, 19, 19, 255)
(19, 19, 19, 255)
(56, 56, 56, 255)
(34, 34, 3

In [42]:
def save_map_data(path, map_dict):
    array_map = [[0] * 40 for _ in range(40)]
    for pos in map_dict.keys():
        array_map[pos[0]][pos[1]] = map_dict[pos]
    with open(path, 'w') as f:
        for line in array_map:
            f.write(','.join(map(str, line)) + '\n')

In [49]:
#time.sleep(2)
#save_map_data("./data/map_array/map_5.txt", get_screen_info())

(106, 110, 137, 255)
(106, 110, 137, 255)
(106, 110, 137, 255)
(106, 110, 137, 255)
(221, 34, 0, 255)
(100, 104, 131, 255)
(221, 34, 0, 255)
(100, 104, 131, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
(221, 34, 0, 255)
